In [186]:
import numpy as np
from tqdm import tqdm

In [195]:
with open("../data/6.txt") as f:
    data = [line.rstrip() for line in f.readlines()]

nr = len(data)
nc = len(data[0])


cardinals = {"^": (-1, 0), ">": (0, 1), "<": (0, -1), "v": (1, 0)}
cardinal_lookup = {"^":0,">":1,"v":2,"<":3}

reverse_lookup = {v:k for k,v in cardinal_lookup.items()}
for k,v in cardinals.items():
    cardinals[k] = np.array(v)

for i in range(nr):
    for j in range(nc):
        if data[i][j] in cardinals:
            start_pos = np.array([i, j])
            start_facing = data[start_pos[0]][start_pos[1]]
            start_direction = cardinals[start_facing]
data = np.array([0 if d=='.' else -1 for line in data for d in line]).reshape((nr,nc))
data[start_pos[0],start_pos[1]] = 1

In [196]:
def valid_step(pos,nr=nr,nc=nc):
    return pos[0]>=0 and pos[1]>=0 and pos[0]<=nr-1 and pos[1]<=nc-1

def simulate_guard_route(pos,data,facing,direction):
    steps = 0
    turns = 0
    rounds = 0
    loop_detector = {f"{pos[0]}_{pos[1]}_{facing}":1}
    while True:
        next_pos = pos + direction
        if not valid_step(next_pos):
            # print(f"guard goes off the map at {next_pos}")
            break
        
        if data[next_pos[0],next_pos[1]] in [0,1]:
            pos = next_pos
            steps += 1
        else: # go right
            facing = reverse_lookup[(turns + 1) % 4]
            direction = cardinals[facing]
            turns += 1
        data[pos[0],pos[1]] = 1
        rounds +=1
        loop_idx = f"{pos[0]}_{pos[1]}_{facing}"
        if loop_idx in loop_detector:
            # print("loop detected!")
            return None
        loop_detector[f"{pos[0]}_{pos[1]}_{facing}"]=1
    return (int((data==1).sum()),rounds,data)


In [ ]:
# part one

In [197]:
answer,num_rounds,new_data = simulate_guard_route(start_pos,np.copy(data),start_facing,start_direction)
print(answer)

4602


In [198]:
# part two
visited_spots = [(int(r),int(c)) for r,c in zip(*np.where(new_data==1))]


In [199]:
loop_obs = 0
for vs in tqdm(visited_spots):
    dc = np.copy(data)
    dc[vs[0],vs[1]] = -1
    resp = simulate_guard_route(start_pos,dc,start_facing,start_direction)
    if not resp:
        loop_obs += 1

    

100%|██████████| 4602/4602 [00:19<00:00, 241.44it/s]


In [ ]:
loop_obs - 1

1704